In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sqlite3
from pprint import pprint as pp
import json
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, Line, LinearAxis, Range1d
import pandas_bokeh
import os

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
conn = sqlite3.connect('../db/11_Jun.db')

In [4]:
lai_df = pd.read_sql("select * from lai", conn, parse_dates=['date_time'])
lai_sorted = lai_df.sort_values('date_time')

In [5]:
n18_lai = lai_sorted[(lai_sorted['georegion'] == 'East 18 North third') & (lai_sorted['vis_moisture'] == 0)]
m18_lai = lai_sorted[(lai_sorted['georegion'] == 'East 18 Middle third') & (lai_sorted['vis_moisture'] == 0)]
s18_lai = lai_sorted[(lai_sorted['georegion'] == 'East 18 South third') & (lai_sorted['vis_moisture'] == 0)]

In [7]:
precip_df = pd.read_csv('../../data/Jan 19 - May 20 precip.csv', parse_dates=['DATE'])
precip_df

,STATION,NAME,DATE,PRCP,PRCP_ATTRIBUTES,SNOW,SNOW_ATTRIBUTES
0,USW00094822,"ROCKFORD GREATER ROCKFORD AIRPORT, IL US",2019-01-01,0.00,"T,,W,2400",0.0,"T,,W"
1,USW00094822,"ROCKFORD GREATER ROCKFORD AIRPORT, IL US",2019-01-02,0.00,"T,,W,2400",0.0,"T,,W"
2,USW00094822,"ROCKFORD GREATER ROCKFORD AIRPORT, IL US",2019-01-03,0.00,",,W,2400",0.0,",,W"
3,USW00094822,"ROCKFORD GREATER ROCKFORD AIRPORT, IL US",2019-01-04,0.00,",,W,2400",0.0,",,W"
4,USW00094822,"ROCKFORD GREATER ROCKFORD AIRPORT, IL US",2019-01-05,0.00,",,W,2400",0.0,",,W"
...,...,...,...,...,...,...,...
494,USW00094822,"ROCKFORD GREATER ROCKFORD AIRPORT, IL US",2020-05-09,0.00,"T,,D,2400",0.0,",,D"
495,USW00094822,"ROCKFORD GREATER ROCKFORD AIRPORT, IL US",2020-05-10,0.25,",,D,2400",0.0,",,D"
496,USW00094822,"ROCKFORD GREATER ROCKFORD AIRPORT, IL US",2020-05-11,0.00,"T,,D,2400",0.0,",,D"
497,USW00094822,"ROCKFORD GREATER ROCKFORD AIRPORT, IL US",2020-05-12,0.00,",,D,2400",0.0,",,D"


In [8]:
precip_df['SMA'] = precip_df.loc[:,['PRCP']].rolling(window=21).mean()
valid_precip = precip_df[precip_df['SMA'] >= 0]

In [11]:
n18_source = ColumnDataSource(n18_lai)
m18_source = ColumnDataSource(m18_lai)
s18_source = ColumnDataSource(s18_lai)
precip_source = ColumnDataSource(valid_precip)

p = figure(x_axis_type="datetime", y_range=(0,5),
          title='LAI and Rain',
          x_axis_label='Date',
          y_axis_label='LAI value')
p.line('date_time', 'mean', source=n18_source, legend_label='E18 North', color='green', line_width=2)
p.line('date_time', 'mean', source=m18_source, legend_label='E18 Middle', color='blue', line_width=2)
p.line('date_time', 'mean', source=s18_source, legend_label='E18 South', color='purple', line_width=2)

p.extra_y_ranges = {"rain": Range1d(start=0, end=1)}
p.line('DATE', 'SMA', source=precip_source, legend_label='Daily rain (in)', color="orange", line_width=2, y_range_name='rain')
p.add_layout(LinearAxis(y_range_name="rain", axis_label='Average daily rainfall'), 'right')

show(p)